In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book(
)
from fastai.vision.all import *
from fastbook import *

path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]

stacked_sevens = torch.stack(seven_tensors).float()/255 #Esto lo que hace es te crea un tensor con 6134 imagenes de 28x28, 
#el otro era una lista de imagenes.
stacked_threes = torch.stack(three_tensors).float()/255


In [2]:
valid_3_tens = torch.stack([tensor(Image.open(o))for o in (path/'valid'/'3').ls()]).float()
#aca busco las imagenes y las stackeo
valid_7_tens = torch.stack([tensor(Image.open(o))for o in (path/'valid'/'7').ls()]).float()


valid_x= torch.cat([valid_7_tens,valid_3_tens]).view(-1,28*28)  
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1).float()

#hasta ahora tenés tu set de validación y ahora tenés que crear los dataloader para despues crear el DataLoaders
#que es el que va a a entrar al learner( el dataloaders contiene a loos 2 dataloader)
valid_x.shape,valid_y.shape

(torch.Size([2038, 784]), torch.Size([2038, 1]))

In [3]:
#el train_x son los set de x, y van con el train_Y, los valid_x con el valid_y
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28) #acá hago mas chico el area y stackeo los 3 
train_y = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1).float() #acá le estamos diciendo que train_y
                               #va a tener un 1 que representa al 3 y un 0 que 
                                #y los 7

        

dl_train=DataLoader(list(zip(train_x,train_y)),bs=64)
dl_valid=DataLoader(list(zip(valid_x,valid_y)),bs=64)
dls= DataLoaders(dl_train,dl_valid) 
# rep
train_x.shape, train_y.shape

(torch.Size([12396, 784]), torch.Size([12396, 1]))

In [4]:
def init_parameters(inp, outp):
     return torch.randn(inp,outp).requires_grad_()

class SimpleModel():
    
    
    def __init__(self,in_features,out_features):
        self.in_f=in_features
        self.out_f=out_features
        self.weights= init_parameters(in_features,out_features)
        self.bias=init_parameters(1,1)
    def predict(self, x):
        return x@self.weights + self.bias

In [5]:
init_parameters(28*28,1).shape


torch.Size([784, 1])

<h1> no encuentro el error, p.grad es none</h1>
<h1> y creo que una vez lo hice andar (no recuerdo como), pero me daba siempre los mismos weights</h1> 

In [6]:
def init_parameters(inp, outp):
     return torch.randn(inp,outp).requires_grad_() #funcion randn

    
class SimpleModel():
     
    def __init__(self,in_features,out_features): #entran 28*28 pixeles y sale 1 salida
        self.in_f=in_features
        self.out_f=out_features
        self.weights= init_parameters(in_features,out_features)
        self.bias=init_parameters(1  ,1)
        self.params =[self.weights,self.bias]
    def forward(self, x):
        return x @ self.weights + self.bias
    
    
def my_loss(predict,target): # si necesito mas elementos puedo hacer una clase como hicimos abajo 
    pred_normalized= predict.sigmoid()
    return (target-pred_normalized).abs().mean()

class MyOptimizer():
    def __init__(self,lr=1e-2):
        self.params= None
        self.lr=lr
    def step(self):
        for p in self.params:
           # print("p.data",p.data);
           # print("soy p.grad",p.grad);
            p.data -=self.lr*p.grad 
    def zero_grad(self):
        for p in self.params:
            p.grad.zero_()

def my_metric(predict,target):
    pred= predict.sigmoid().round()
    
    return (target==pred).float().mean()
    
class MyLearner():
        def __init__(self,dls,model,loss,opt,metric):
            self.dls=dls
            self.model=model
            self.loss=loss
            self.opt=opt
            self.metric=metric
            opt.params=model.params
            
        def fit(self,epochs): #cuando le decis que corra dentro de una epoch, ahora estas interando sobre las mismas
            #imagenes varias veces
            for e in range(epochs):
                for train_x, train_y in dls.train:
                    y_predict=model.forward(train_x) 
                    loss=self.loss(y_predict,train_y)
                    self.opt.step()
                    self.opt.zero_grad()    
                   # print("yo soy params",model.params[0][0])
                for x_train, y_train in dls.valid:   #pass #esto es cuando nosabes que quiere hacer
                    y_predict =model.forward(valid_x)
                    accuracy= my_metric(y_predict,valid_y)
                    print(accuracy)
                    print(y_predict)
                    print(valid_y)
                    #c muestra la accuraccy
        #def predict(self,x):
         #    print(model.forward.round())
        

In [7]:
init_parameters(1,2)


tensor([[ 0.3472, -0.2762]], requires_grad=True)

In [10]:
model=SimpleModel(28*28,1)
model.params
Learner=MyLearner(dls,model,my_loss  ,MyOptimizer(lr=0.1),my_metric)
Learner.fit(3)

p.data tensor([[-1.1608e+00],
        [-1.0227e+00],
        [-5.0643e-01],
        [ 7.8016e-01],
        [-3.3962e-02],
        [ 4.1387e-01],
        [-8.5584e-01],
        [ 9.0406e-01],
        [-2.3876e-01],
        [-2.6241e-01],
        [-5.8329e-01],
        [-7.2831e-01],
        [ 3.9296e-01],
        [-1.3158e+00],
        [-8.2709e-01],
        [ 1.5608e+00],
        [ 1.4242e+00],
        [-6.2004e-01],
        [-8.9991e-01],
        [ 4.7916e-01],
        [-2.4811e-01],
        [-1.3093e+00],
        [-2.9196e-01],
        [ 4.5644e-01],
        [ 4.6268e-01],
        [ 2.4934e-01],
        [-1.7062e-01],
        [-6.1410e-01],
        [ 5.3551e-01],
        [-1.6644e+00],
        [-1.5447e-01],
        [-1.6013e+00],
        [ 1.2254e+00],
        [-1.1865e+00],
        [ 4.8605e-01],
        [-4.0234e-01],
        [ 1.1986e-01],
        [-1.7284e-01],
        [ 8.8662e-01],
        [-8.6114e-01],
        [-8.5292e-02],
        [ 9.5743e-01],
        [ 4.5270e-01],
    

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [9]:
weights.grad.mean(),bias.grad

NameError: name 'weights' is not defined

In [11]:
    
class SimpleModel():
     
    def __init__(self,in_features,out_features): #entran 28*28 pixeles y sale 1 salida
        self.in_f=in_features
        self.out_f=out_features
        self.weights= torch.randn(28*28,1).requires_grad_()
        self.bias=torch.randn(1,1).requires_grad_()
        self.params =[self.weights,self.bias]
    def forward(self, x):
        return x @ self.weights + self.bias

In [12]:
model=SimpleModel(28*28,1)

#funcion randn
def calc_grad(train_x, train_y, model):
    for e in range(1):
        print("soy el peso 0-1",model.weights[0][0])
        preds = model.forward(train_x)
        print(preds.shape)
        print(train_y.shape)
        loss = my_loss(preds, train_y)
        print(loss.backward())
        print("soy el peso 0",model.weights[0][0])
        model.weights.data -= 1e-3 * model.weights.grad
        model.weights.grad.zero_()
        print("soy el peso 0+",model.weights[0][0])

def my_loss(predict,target): # si necesito mas elementos puedo hacer una clase como hicimos abajo 
    pred_normalized= predict.sigmoid()
    return (target-pred_normalized).abs().mean()

calc_grad(train_x, train_y, model)



soy el peso 0-1 tensor(1.9996, grad_fn=<SelectBackward>)
torch.Size([12396, 1])
torch.Size([12396, 1])
None
soy el peso 0 tensor(1.9996, grad_fn=<SelectBackward>)
soy el peso 0+ tensor(1.9996, grad_fn=<SelectBackward>)
